- A graph is a mathematical construct where we have a set of object (called nodes) in which some pairs are related.

- "We call each of the nodes a _vertex_ and each of the connections an _edge_.

- Graphs help us to think abstractly about a problem.

# Building a graph framework

We will build an edge class that is flexible.

Note: The __Edge__ class uses a new feature in Python 3.7: dataclasses. A class marked with the decorator @dataclass saves us the trouble of creating an __init()__ method that instantiates instance variables for any variables declared with type annotations in the class's body.

In [1]:
from __future__ import annotations
from dataclasses import dataclass

@dataclass
class Edge:
    u: int # the 'from' vertex
    v: int # the 'to' vertex
    
    # the reversed method is meant to return an Edge that travels the opposite direction
    def reversed(self) -> Edge:
        return Edge(self.v, self.u)
    
    def __str__(self) -> str:
        return f'{self.u} -> {self.u}'

## Graph class

In [2]:
from typing import TypeVar, Generic, List, Optional

V = TypeVar('V') # type of the vertices in the graph

# We initialise a list of vertices with edges to be added later
class Graph(Generic[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[Edge]] = [[] for _ in vertices] #initialised to a list of blank lists
        
    @property
    def vertex_count(self) -> int:
        return len(self._vertices) # number of vertices

    @property
    def edge_count(self) -> int:
        return sum(map(len, self._edges))

    # add a vertex to the graph and return its index
    def add_vertex(self, vertex: V)-> int:
        self._vertices.append(vertex)
        self._edges.append([]) # add empty list for containing edges
        return self.vertex_count - 1 # return index of added vertex

    # This is an undirected graph,
    # so we always add edges in both directions
    def add_edge(self, edge: Edge) -> None:
        self._edges[edge.u].append(edge)
        self._edges[edge.v].append(edge.reversed())
    
    # Add an edge using vertex indices (convenience method)
    def add_edge_by_indices(self, u: int, v: int) -> None:
        edge: Edge = Edge(u, v)
        self.add_edge(edge)
    
    # Add an edge by looking up (existing) vertex indices (convenience method)
    def add_edge_by_vertices(self, first: V, second: V) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v)
    
    # Find the vertex at a specific index
    def vertex_at(self, index: int) -> V:
        return self._vertices[index]
    
    # Find the index of a vertex in the graph
    def index_of(self, vertex: V) -> int:
        return self._vertices.index(vertex)

    # Find the vertices that a vertex t some index is connected to:
    def neighbors_for_index(self, index: int) -> List[V]:
        return list(map(self.vertex_at, [e.v for e in self._edges[index]]))

    # Look up a vertice's index and find its neighbours (convience method)
    def neighbors_for_vertex(self, vertex: V) -> List[V]:
        return self.neighbors_for_index(self.index_of(vertex))
    
    # Return all of the edges associated with a vertex at some index
    def edges_for_index(self, index: int) -> List[Edge]:
        return self._edges[index]

    # Return all of the edges associated with a vertex at some index
    def edges_for_vertex(self, vertex: V) -> List[Edge]:
        return self.edges_for_index(self.index_of(vertex))

    # make sure it is easy to pretty-print a Graph
    def __str__(self) -> str:
        desc: str = ''
        for i in range(self.vertex_count):
            desc += f'{self.vertex_at(i)} -> {self.neighbors_for_index(i)} \n'
        return desc

# Working with Edge and Graph (4.2.1)

In [3]:
# test basic Graph construction
city_graph: Graph[str] = Graph(['Seattle', 'San Francisco', 'Los Angeles', 
                                'Riverside', 'Phoenix', 'Chicago', 'Boston',
                                'New York','Atlanta', 'Miami', 'Dallas', 
                                'Houston', 'Detroit', 'Philadelphia',
                                'Washington'])
city_graph.add_edge_by_vertices('Seattle', 'Chicago')
city_graph.add_edge_by_vertices('Seattle', 'San Francisco')
city_graph.add_edge_by_vertices('San Francisco', 'Riverside')
city_graph.add_edge_by_vertices('San Francisco', 'Los Angeles')
city_graph.add_edge_by_vertices('Los Angeles', 'Riverside')
city_graph.add_edge_by_vertices('Los Angeles', 'Phoenix')
city_graph.add_edge_by_vertices('Riverside', 'Phoenix')
city_graph.add_edge_by_vertices('Riverside', 'Chicago')
city_graph.add_edge_by_vertices('Phoenix', 'Dallas')
city_graph.add_edge_by_vertices('Phoenix', 'Houston')
city_graph.add_edge_by_vertices('Dallas', 'Chicago')
city_graph.add_edge_by_vertices('Dallas', 'Atlanta')
city_graph.add_edge_by_vertices('Dallas', 'Houston')
city_graph.add_edge_by_vertices('Houston', 'Atlanta')
city_graph.add_edge_by_vertices('Houston', 'Miami')
city_graph.add_edge_by_vertices('Atlanta', 'Chicago')
city_graph.add_edge_by_vertices('Atlanta', 'Washington')
city_graph.add_edge_by_vertices('Atlanta', 'Miami')
city_graph.add_edge_by_vertices('Miami', 'Washington')
city_graph.add_edge_by_vertices('Chicago', 'Detroit')
city_graph.add_edge_by_vertices('Detroit', 'Boston')
city_graph.add_edge_by_vertices('Detroit', 'Washington')
city_graph.add_edge_by_vertices('Detroit', 'New York')
city_graph.add_edge_by_vertices('Boston', 'New York')
city_graph.add_edge_by_vertices('New York', 'Philadelphia')
city_graph.add_edge_by_vertices('Philadelphia', 'Washington')
print(city_graph)

Seattle -> ['Chicago', 'San Francisco'] 
San Francisco -> ['Seattle', 'Riverside', 'Los Angeles'] 
Los Angeles -> ['San Francisco', 'Riverside', 'Phoenix'] 
Riverside -> ['San Francisco', 'Los Angeles', 'Phoenix', 'Chicago'] 
Phoenix -> ['Los Angeles', 'Riverside', 'Dallas', 'Houston'] 
Chicago -> ['Seattle', 'Riverside', 'Dallas', 'Atlanta', 'Detroit'] 
Boston -> ['Detroit', 'New York'] 
New York -> ['Detroit', 'Boston', 'Philadelphia'] 
Atlanta -> ['Dallas', 'Houston', 'Chicago', 'Washington', 'Miami'] 
Miami -> ['Houston', 'Atlanta', 'Washington'] 
Dallas -> ['Phoenix', 'Chicago', 'Atlanta', 'Houston'] 
Houston -> ['Phoenix', 'Dallas', 'Atlanta', 'Miami'] 
Detroit -> ['Chicago', 'Boston', 'Washington', 'New York'] 
Philadelphia -> ['New York', 'Washington'] 
Washington -> ['Atlanta', 'Miami', 'Detroit', 'Philadelphia'] 



# Revisiting Breadth-first search (BFS)

“What is the shortest path between Boston and Miami?”

We can reuse the generic code in Chapter 2.

Note: https://stackoverflow.com/questions/4383571/importing-files-from-different-folder

In [4]:
import sys
sys.path.append('../Chapter 2/')
from generic_search import bfs, Node, node_to_path

# bfs takes 3 arguments: starting point, a callable that returns a boolean value, and a callable that returns
# a list of lists
bfs_result: Optional[Node[V]] = bfs('Boston', lambda x: x == 'Miami', city_graph.neighbors_for_vertex)

if bfs_result is None:
    print('No solution found using breadth first search!')
else:
    path: List[V] = node_to_path(bfs_result)
    print('Path from Boston to Miami \n')
    print(path)

Path from Boston to Miami 

['Boston', 'Detroit', 'Washington', 'Miami']


# Minimising the cost of building the network

In the previous breadth first search, we looked at the path with the fewest steps. However, in practice, edges have weights representing some kind of cost of travelling from one vertex to another. In the current example, we will add weights that represent the distances in miles between the cities (vertices).

To handle weights, we need a subclass of __Edge__ - __WeightedEdge__ and a subclass of __Graph__ - __WeightedGraph__.

In [5]:
from __future__ import annotations
from dataclasses import dataclass

@dataclass
class WeightedEdge(Edge):
    weight: float
    
    def reversed(self) -> WeightedEdge:
        return WeightedEdge(self.v, self.u, self.weight)
    
    # include comparison dunder method
    def __lt__(self, other: WeightedEdge) -> bool:
        return self.weight < other.weight
    
    def __str__(self) -> str:
        return f'{self.u} {self.v} {self.v}'

In [6]:
from typing import TypeVar, Generic, List, Tuple

V = TypeVar('V') # type of the vertices in the graph

# Generic[V] indicates the class can take any data types
# The second argument Graph[V] indicates it is a subclass
class WeightedGraph(Generic[V], Graph[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[WeightedEdge]] = [[] for _ in vertices]
    
    def add_edge_by_indices(self, u: int, v: int, weight: float) -> None:
        edge: WeightedEdge = WeightedEdge(u, v, weight)
        self.add_edge(edge) # call superclass version
            
    def add_edge_by_vertices(self, first: V, second: V, weight: float) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v, weight)
        
    def neighbors_for_index_with_weights(self, index: int) -> List[Tuple[V, float]]:
        distance_tuples: List[Tuple[V, float]] = []
        for edge in self.edges_for_index(index):
            distance_tuples.append((self.vertex_at(edge.v), edge.weight))
        return distance_tuples
    
    def __str__(self) -> str:
        desc: str = ''
        for i in range(self.vertex_count):
            desc += f'{self.vertex_at(i)} -> {self.neighbors_for_index_with_weights(i)} \n'
        return desc

In [7]:
city_graph2: WeightedGraph[str] = WeightedGraph(['Seattle', 'San Francisco',
                                                 'Los Angeles', 'Riverside', 
                                                 'Phoenix', 'Chicago', 'Boston',
                                                 'New York', 'Atlanta', 'Miami', 
                                                 'Dallas', 'Houston', 'Detroit',
                                                 'Philadelphia', 'Washington'])
    
city_graph2.add_edge_by_vertices('Seattle', 'Chicago', 1737)
city_graph2.add_edge_by_vertices('Seattle', 'San Francisco', 678)
city_graph2.add_edge_by_vertices('San Francisco', 'Riverside', 386)
city_graph2.add_edge_by_vertices('San Francisco', 'Los Angeles', 348)
city_graph2.add_edge_by_vertices('Los Angeles', 'Riverside', 50)
city_graph2.add_edge_by_vertices('Los Angeles', 'Phoenix', 357)
city_graph2.add_edge_by_vertices('Riverside', 'Phoenix', 307)
city_graph2.add_edge_by_vertices('Riverside', 'Chicago', 1704)
city_graph2.add_edge_by_vertices('Phoenix', 'Dallas', 887)
city_graph2.add_edge_by_vertices('Phoenix', 'Houston', 1015)
city_graph2.add_edge_by_vertices('Dallas', 'Chicago', 805)
city_graph2.add_edge_by_vertices('Dallas', 'Atlanta', 721)
city_graph2.add_edge_by_vertices('Dallas', 'Houston', 225)
city_graph2.add_edge_by_vertices('Houston', 'Atlanta', 702)
city_graph2.add_edge_by_vertices('Houston', 'Miami', 968)
city_graph2.add_edge_by_vertices('Atlanta', 'Chicago', 588)
city_graph2.add_edge_by_vertices('Atlanta', 'Washington', 543)
city_graph2.add_edge_by_vertices('Atlanta', 'Miami', 604)
city_graph2.add_edge_by_vertices('Miami', 'Washington', 923)
city_graph2.add_edge_by_vertices('Chicago', 'Detroit', 238)
city_graph2.add_edge_by_vertices('Detroit', 'Boston', 613)
city_graph2.add_edge_by_vertices('Detroit', 'Washington', 396)
city_graph2.add_edge_by_vertices('Detroit', 'New York', 482)
city_graph2.add_edge_by_vertices('Boston', 'New York', 190)
city_graph2.add_edge_by_vertices('New York', 'Philadelphia', 81)
city_graph2.add_edge_by_vertices('Philadelphia', 'Washington', 123)

print(city_graph2)

Seattle -> [('Chicago', 1737), ('San Francisco', 678)] 
San Francisco -> [('Seattle', 678), ('Riverside', 386), ('Los Angeles', 348)] 
Los Angeles -> [('San Francisco', 348), ('Riverside', 50), ('Phoenix', 357)] 
Riverside -> [('San Francisco', 386), ('Los Angeles', 50), ('Phoenix', 307), ('Chicago', 1704)] 
Phoenix -> [('Los Angeles', 357), ('Riverside', 307), ('Dallas', 887), ('Houston', 1015)] 
Chicago -> [('Seattle', 1737), ('Riverside', 1704), ('Dallas', 805), ('Atlanta', 588), ('Detroit', 238)] 
Boston -> [('Detroit', 613), ('New York', 190)] 
New York -> [('Detroit', 482), ('Boston', 190), ('Philadelphia', 81)] 
Atlanta -> [('Dallas', 721), ('Houston', 702), ('Chicago', 588), ('Washington', 543), ('Miami', 604)] 
Miami -> [('Houston', 968), ('Atlanta', 604), ('Washington', 923)] 
Dallas -> [('Phoenix', 887), ('Chicago', 805), ('Atlanta', 721), ('Houston', 225)] 
Houston -> [('Phoenix', 1015), ('Dallas', 225), ('Atlanta', 702), ('Miami', 968)] 
Detroit -> [('Chicago', 238), ('Bos

### Finding the minimum spanning tree

"A tree is a special kind of graph that has one, and only one, path between any two vertices. This implies that there are no cycles in a tree (which is sometimes called acyclic). A cycle can be thought of as a loop: if it is possible to traverse a graph from a starting vertex, never repeat any edges, and get back to the same starting vertex, then it has a cycle. Any graph that is not a tree can become a tree by pruning edges.

A connected graph is a graph that has some way of getting from any vertex to any other vertex. (All of the graphs we are looking at in this chapter are connected.) A spanning tree is a tree that connects every vertex in a graph. A minimum spanning tree is a tree that connects every vertex in a weighted graph with the minimum total weight (compared to other spanning trees). For every weighted graph, it is possible to efficiently find its minimum spanning tree. "

We will create another priority queue using the heap queue (__heapq__) methods to keep our priority queue data structure. The data passed in the list must have the __ __lt__ __ method defined for it to work.

In [8]:
from typing import TypeVar, Generic, List
from heapq import heappush, heappop

T = TypeVar('T')

class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []

    @property
    def empty(self) -> bool:
        return not self._container  # not is true for empty container

    def push(self, item: T) -> None:
        heappush(self._container, item)  # in by priority

    def pop(self) -> T:
        return heappop(self._container)  # out by priority

    def __repr__(self) -> str:
        return repr(self._container)

We also want a way to find the total weight of the edges in a path:

In [9]:
from typing import TypeVar, List, Optional


V = TypeVar('V') # type of the vertices in the graph
WeightedPath = List[WeightedEdge] # type alias for paths

def total_weight(wp: WeightedPath) -> float:
    return sum([e.weight for e in wp])

## Jarnik's Algorithm

Jarnik's Algorithm works by dividing a graph into two parts:

1) the vertices in the still-being-assembed minimum spanning tree (MST).

2) the vertices not yet in the minimum spanning tree.

These are the steps:

1. Pick a starting vertex to be in the MST.
2. Find the lowest-weight edge (minimum edge) connecting the MST to the vertices not yet in the MST.
3. Add the vertex at the end of that minimum edge to the MST.
4. Repeat steps 2 and 3 until every vertex in the graph is in the MST.

"To run Jarník’s algorithm efficiently, a priority queue is used. Every time a new vertex is added to the minimum spanning tree, all of its outgoing edges that link to vertices outside the tree are added to the priority queue. The lowest-weight edge is always popped off the priority queue, and the algorithm keeps executing until the priority queue is empty. This ensures that the lowest-weight edges are always added to the tree first. Edges that connect to vertices already in the tree are ignored when they are popped. 

Warning

Jarnik's algorithm will not necessarily work correctly in a graph with directed edges. It also will not work in a graph that is not connected.
"

In [10]:
def mst(wg: WeightedGraph[V], start: int = 0) -> Optional[WeightedPath]:
    if start >(wg.vertex_count - 1) or start < 0:
        return None
    result: WeightedPath = [] # holds the final MST
    pq: PriorityQueue[WeightedEdge] = PriorityQueue() # for adding edges of the new vertex
    visited: [bool] = [False] * wg.vertex_count # where we've been
    # This could also have been accomplished with a Set, similar to explored in bfs()
    
    def visit(index: int):
        visited[index] = True # mark as visited
        for edge in wg.edges_for_index(index):
            # add all edges coming from here to pq
            if not visited[edge.v]:
                pq.push(edge)
    
    # the first vertex is where everything begins
    # visit() is an inner convenience function that marks 
    # a vertex as visited and adds all of its edges that 
    # connect to vertices not yet visited to pq
    visit(start)

    while not pq.empty: # keep going while there are edges to process
        edge = pq.pop()
        if visited[edge.v]:
            continue # don't ever revisit
        # this s the current smallest, so add it to solution
        result.append(edge)
        visit(edge.v) # visit where this connects
    
    return result

def print_weighted_path(wg: WeightedGraph, wp: WeightedPath) -> None:
    for edge in wp:
        print(f'{wg.vertex_at(edge.u)} {edge.weight}> {wg.vertex_at(edge.v)}')
        
    print(f'Total Weight: {total_weight(wp)}')

Solving __city_graph2__

In [11]:
result: Optional[WeightedPath] = mst(city_graph2)
if result is None:
    print('No solution found!')
else:
    print_weighted_path(city_graph2, result)

Seattle 678> San Francisco
San Francisco 348> Los Angeles
Los Angeles 50> Riverside
Riverside 307> Phoenix
Phoenix 887> Dallas
Dallas 225> Houston
Houston 702> Atlanta
Atlanta 543> Washington
Washington 123> Philadelphia
Philadelphia 81> New York
New York 190> Boston
Washington 396> Detroit
Detroit 238> Chicago
Atlanta 604> Miami
Total Weight: 5372


"In other words, this is the cumulatively shortest collection of edges that connects all of the MSAs in the weighted graph. The minimum length of track needed to connect all of them is 5,372 miles."